### Loading libraries and data.

In [ ]:
# For Google Collab to choose a newer version of TensorFlow
%tensorflow_version 2.x

In [ ]:
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from skimage.color import rgb2hed
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from scipy.stats import entropy
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm import SVR, LinearSVR
import statsmodels.api as sm
from sklearn import preprocessing

In [ ]:
# tensorflow imports
from __future__ import absolute_import, division, print_function
import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
import os
import tensorflow as tf
from tensorflow import keras
from keras.constraints import max_norm
tf.__version__

In [ ]:
# checkin if GPU is on
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# GOOGLE COLLAB ONLY

# For Google Collab to mount the drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Import and preprare the data
D = h5py.File('/content/drive/My Drive/breast.h5', 'r') 
X,Y,P = D['images'],np.array(D['counts']),np.array(D['id'])

The challenge webpage indicates that the area equal to 16px of a border around the edges was not used to count the number of  lymphocytes. Thus, I have decided to trim this area.

In [ ]:
# trimming the are of the images equal to 16px
X = X[:,16:283,16:283,:]

In [ ]:
# GOOGLE COLLAB ONLY - I HAD ISSUES TO IMPORT DATA WITH GPU

# For Google Collab only - h5py with GPU does not work
#X = np.load('/content/drive/My Drive/Colab Notebooks/X_data.npy')
#Y = np.load('/content/drive/My Drive/Colab Notebooks/Y_data.npy')

## Question 1


### i) Data breakdown

There are 5841 training examples and 1563 testing examples in the data. 

In [ ]:
# number of train examples
np.sum(P<14)

In [ ]:
# number of test examples
np.sum(P>=14)

### ii) Showing Images

In [ ]:
# showing images
for i in range(0,5): 
    plt.imshow(X[i])
    plt.show()

In [ ]:
# Seeing respective count values
Y[0:5]

The images show tissue with brown and blue cells on a light background. The brown cells are the cells of interest, however, how "easy" it is to count them varies. For example, the last image on display clearly shows that there are 6 lymphocytes whilst all the other images on display portray more ambiguous cases. It seems that the medical specialists were only counting the big and rather bold cells ignoring the small and bright ones. However, as the penultimate image with count 5 shows, this might not be a strong rule.

Furthermore, the challenge creators have added zero counts images into the dataset too. 

In [ ]:
plt.imshow(X[21,:,:,:])

This image, for the human eye, is very obviously different and has no brown cells. Yet, some slightly darker patterns within an image might make the differentiation between two types of images difficult.

### iii) Histogram

In [ ]:
# crude histogram
plt.hist(Y, color = "sandybrown")
plt.title("Histogram of counts")
plt.show()

The histogram above shows a strong right skew. This indicates that the majority of the values are resting within 0 to 10 cells as well as that there are some outliers which need to be investigated. To investigate the outliers, I first want to see how many of them there are.

In [ ]:
# how many big values - might impose a problem.
np.sum(Y>40)

In [ ]:
# are there any with more than 200?
np.sum(Y>200)

There are only 5 values which are larger than 40 and none of the values are larger than 200 (thus I will exclude larger than 200 bin from the histogram required). The required histogram is presented bellow. 

In [ ]:
# histogram by the (roughly) required bins. (I know there are no >200)
(histogram_counts, bins, _) = plt.hist(Y, bins = [0,1,6,11,21,51,201], color = "sandybrown")
plt.title("Histogram of counts")
plt.show()

In [ ]:
np.set_printoptions(suppress=True)
print(histogram_counts)
print(bins)

The majority of the values lie between 0 to 51. There is only one value greater than 51. I want to examine it closer.

In [ ]:
Y[Y>50]

Only one image has a cell count of 70. This sounds like an enormous number and I want to check the image with this count. It is important because if this is a mistake and not a real count, it might influence the models too much.

In [ ]:
print(np.where(Y==70))
plt.imshow(X[2892])
plt.show()

This image truly has a lot of brown cells, thus I am leaving this observation untouched.

### iv) Pre-processing

To save time and stop my RAMs from crashing I have decided to transform the pre-processed brown layer data to int8 format from the original float32

In [ ]:
np.shape(X)

In [ ]:
hed_data = rgb2hed(X[:1000,:,:,:])[:,:,:,2]
hed_data = (hed_data * 128).astype("int8")

In [ ]:
hed_data_2 = rgb2hed(X[1000:2000])[:,:,:,2]
hed_data_2 = (hed_data_2 * 128).astype("int8")

In [ ]:
hed_data = np.vstack((hed_data, hed_data_2))

In [ ]:
hed_data_2 = rgb2hed(X[2000:3000])[:,:,:,2]
hed_data_2 = (hed_data_2 * 128).astype("int8")

In [ ]:
hed_data = np.vstack((hed_data, hed_data_2))

In [ ]:
hed_data_2 = rgb2hed(X[3000:4000])[:,:,:,2]
hed_data_2 = (hed_data_2 * 128).astype("int8")

In [ ]:
hed_data = np.vstack((hed_data, hed_data_2))

In [ ]:
hed_data_2 = rgb2hed(X[4000:5000])[:,:,:,2]
hed_data_2 = (hed_data_2 * 128).astype("int8")

In [ ]:
hed_data = np.vstack((hed_data, hed_data_2))

In [ ]:
hed_data_2 = rgb2hed(X[5000:6000])[:,:,:,2]
hed_data_2 = (hed_data_2 * 128).astype("int8")

In [ ]:
hed_data = np.vstack((hed_data, hed_data_2))

In [ ]:
hed_data_2 = rgb2hed(X[6000:])[:,:,:,2]
hed_data_2 = (hed_data_2 * 128).astype("int8")

In [ ]:
hed_data = np.vstack((hed_data, hed_data_2))
del(hed_data_2)
# saving data for later use
np.save("hed_data", hed_data)

Showing the pre-processed images.

In [ ]:
# showing the pre-processed images
cmap_hema = LinearSegmentedColormap.from_list('mycmap', ['white', 'navy'])
cmap_dab = LinearSegmentedColormap.from_list('mycmap', ['white', 'saddlebrown'])
cmap_eosin = LinearSegmentedColormap.from_list('mycmap', ['darkviolet', 'white'])

for i in range(0,3):
  fig, axes = plt.subplots(1, 2, figsize=(7, 6), sharex=True, sharey=True)
  ax = axes.ravel()
  ax[0].imshow(X[i,])
  ax[0].set_title("Original image")

  ax[1].imshow(hed_data[i,], cmap=cmap_dab)
  ax[1].set_title("DAB")

### v) Scatter Plot

Creating data for an average "brownness" scatter plot.

In [ ]:
average_brown = []
for i in range(0, np.shape(hed_data)[0]):
    average_brown.append(np.mean(hed_data[i]))
    
average_brown = np.asarray(average_brown)  

In [ ]:
plt.scatter(average_brown, Y, color = "saddlebrown")
plt.title("Average brown vs cell count")
plt.show()

The scatter plot above shows a strong positive correlation. The higher the number of cells - the more "brownness" there is. When it comes to zero count data, we see that those images can roughly be of any degree of "brownness". This almost surely will be a problem and the models are likely to struggle to recognise zero count images. Furthermore, the scatter plot also indicates that this covariate might struggle with smaller count data. Looking into pictures, we see that the size of the brown cells varies, and if they are smaller it might appear that the image is less brown, but the count of the cells might, nonetheless, be non-zero. Hence, this is a problem and we need to look for additional covariates.

### vi) Number of images per patient

In [ ]:
np.unique(P, return_counts = True)

The number of images per patient is not even, yet I do not think it is a problem. We are not looking into person specific problem, but instead, we want to develop a pipeline that could predict the number of cells given any image. We must not rely on the prior knowledge - the prior cell count of the person. If we were to include covariates such as age, gender, lifestyle, etc. to improve our analysis then the individual counts might be more important.

### vii) Performance measure

I have chosen MSE to be the performance measure for this problem. There are two reasons behind it. First, MSE is nicely differentiable. Since I will be implementing Multilayer Perceptron and Convolutional Neural Networks, having a loss function with an easy to compute gradient, can make the problem slightly easier to solve as opposed to using, for example, mean absolute error. Secondly, MSE does not assume that the error of 10 is twice as bad as the error of 5. Instead, it assumes the error of 10 is four times as bad as the error of 5. In this specific context of leukocytes count, where the result of an error could be a wrong treatment for a patient, I believe we need to penalize the errors harsher. Nonetheless, MSE has its flaws too. For example, the training data has some big outliers (cell counts of more than 40, 70, thus it is unlikely that the model will predict them very acurately. As a result, MSE would explode in the presence of such an error. In addition, MSE is not interpretable in the units of measure. Luckily, this is an easily solvable problem if we take the square root of MSE. Thus, for most of the problems, I will be reporting RMSE only, yet when I will be conducting neural networks, I will use MSE as a loss functions behind the scenes but report RMSE instead owing to its interpretability.

## Question 2

### i) Extracting features

### a. Averages

Initially, I start by extracting the average colour of all the layers. Average "brownness" has been extracted before in Question 1, thus I will not do it again here. I plot scatter plots of each measure below as well as report the correlation coefficients of each feature will the cell counts. 

In [ ]:
# average red
average_red = []
for i in range(0, np.shape(X)[0]):
    average_red.append(np.mean(X[i,:,:,0]))
    
# transforming into array    
average_red = np.asarray(average_red)    
#np.save("average_red", average_red)

In [ ]:
# average green
average_green = []
for i in range(0, np.shape(X)[0]):
    average_green.append(np.mean(X[i,:,:,1]))
# transforming into array     
average_green = np.asarray(average_green)    
#np.save("average_green", average_green)

In [ ]:
# average blue
average_blue = []
for i in range(0, np.shape(X)[0]):
    average_blue.append(np.mean(X[i,:,:,2]))
average_blue = np.asarray(average_blue)
#np.save("average_blue", average_blue)

In [ ]:
plt.scatter(average_red, Y, color="firebrick")
plt.title("Average red vs cell count")
plt.show()
plt.scatter(average_green, Y, color="olivedrab")
plt.title("Average green vs cell count")
plt.show()
plt.scatter(average_blue, Y, color="royalblue")
plt.title("Average blue vs cell count")
plt.show()
plt.scatter(average_brown, Y, color="saddlebrown")
plt.title("Average brown ves cell count")
plt.show()

In [ ]:
print("Browness and counts correlation:", np.corrcoef(average_brown, Y)[0,1],
     "Blueness and counts correlation:", np.corrcoef(average_blue, Y)[0,1],
     "Redness and counts correlation:", np.corrcoef(average_red, Y)[0,1],
    "Greeness and counts correlation:", np.corrcoef(average_green, Y)[0,1])

Brownness, as discussed before, seems like a reasonably good predictor. The correlation coefficient is larger in absolute terms than that of the other three channels. Apart from that, all the channels showed some degree of codependency with the cell counts with blueness being the strongest based on the correlation coefficient among the three RGB layers. This is expected considering that the blue cells are those which "compete" for space with the brown ones. Overall, I believe that all of these four measures are worth keeping in at least for the initial regressions.

### b. Variance

For the variance, I complete the same analysis as I did for the average colour of a filter data.

In [ ]:
variance_brown = []
for i in range(0, np.shape(hed_data)[0]):
    variance_brown.append(np.var(hed_data[i]))
    
variance_brown = np.asarray(variance_brown)
#np.save("variance_brown", variance_brown)

In [ ]:
variance_red = []
for i in range(0, np.shape(X)[0]):
    variance_red.append(np.var(X[i,:,:,0]))
    
variance_red = np.asarray(variance_red)
#np.save("variance_red", variance_red)

In [ ]:
variance_green = []
for i in range(0, np.shape(X)[0]):
    variance_green.append(np.var(X[i,:,:,1]))
    
variance_green = np.asarray(variance_green)
#np.save("variance_green", variance_green)

In [ ]:
variance_blue = []
for i in range(0, np.shape(X)[0]):
    variance_blue.append(np.var(X[i,:,:,2]))
    
variance_blue = np.asarray(variance_blue)
#np.save("variance_blue", variance_blue)

In [ ]:
plt.scatter(variance_red, Y, color="firebrick")
plt.title("Variance of red vs cell count")
plt.show()
plt.scatter(variance_green, Y, color="olivedrab")
plt.title("Variance of green vs cell count")
plt.show()
plt.scatter(variance_blue, Y, color="royalblue")
plt.title("Variance of blue vs cell count")
plt.show()
plt.scatter(variance_brown, Y, color="saddlebrown")
plt.title("Variance of brown vs cell count")
plt.show()

In [ ]:
print("Variance of brown and counts correlation:", np.corrcoef(variance_brown, Y)[0,1],
     "Variance of blue and counts correlation:", np.corrcoef(variance_blue, Y)[0,1],
     "Variance of red and counts correlation:", np.corrcoef(variance_red, Y)[0,1],
    "Variance of green and counts correlation:", np.corrcoef(variance_green, Y)[0,1])

From the scatterplots, we can see that the variance feature is a worse predictor than the average shade. The issue is that the variance covers almost all possible value space for the zero count data exceeding that of the non-zero count data. Nonetheless, the variance of brownness was the strongest predictor followed by the blueness again. Green and red variances show correlation of 0.25 and 0.16 respectively, thus I consider oomitting these variables out of the regressions.

### c. Entropy

As before, the calculation of entropy will be followed by the scatter plots and the correlation coefficient with the cell count data.

In [ ]:
# entropy red
entropy_red = []
for i in range(0, np.shape(X)[0]):
    # need to reshape the image to get a single value of entropy
    entropy_red.append(entropy(np.reshape(X[i,:,:,0], (267*267, ))))
    
entropy_red = np.asarray(entropy_red)    
#np.save("entropy_red", entropy_red)

In [ ]:
# entropy green
entropy_green = []
for i in range(0, np.shape(X)[0]):
    # need to reshape the image to get a single value of entropy
    entropy_green.append(entropy(np.reshape(X[i,:,:,1], (267*267, ))))
    
entropy_green = np.asarray(entropy_green)    
#np.save("entropy_green", entropy_green)

In [ ]:
# entropy blue
entropy_blue = []
for i in range(0, np.shape(X)[0]):
    # need to reshape the image to get a single value of entropy
    entropy_blue.append(entropy(np.reshape(X[i,:,:,2], (267*267, ))))
    
entropy_blue = np.asarray(entropy_blue)    
#np.save("entropy_blue", entropy_blue)

In [ ]:
# entropy brown
entropy_brown = []
for i in range(0, np.shape(hed_data)[0]):
    # need to reshape the image to get a single value of entropy
    entropy_brown.append(entropy(np.reshape(hed_data[i], (267*267, ))))
    
entropy_brown = np.asarray(entropy_brown)    
#np.save("entropy_brown", entropy_brown)

In [ ]:
plt.scatter(entropy_red, Y, color="firebrick")
plt.title("Red entropy vs cell count")
plt.show()
plt.scatter(entropy_green, Y, color="olivedrab")
plt.title("Green entropy vs cell count")
plt.show()
plt.scatter(entropy_blue, Y, color="royalblue")
plt.title("Blue entropy vs cell count")
plt.show()
plt.scatter(entropy_brown, Y, color="saddlebrown")
plt.title("Brown entropy vs cell count")
plt.show()

In [ ]:
print("Brown entropy and counts correlation:", np.corrcoef(entropy_brown, Y)[0,1],
     "Blue entropy and counts correlation:", np.corrcoef(entropy_blue, Y)[0,1],
     "Red entropy and counts correlation:", np.corrcoef(entropy_red, Y)[0,1],
    "Green entropy and counts correlation:", np.corrcoef(entropy_green, Y)[0,1])

Entropy suffers from the same problem as the variance - zero counts cover all possible entropy value space. As before, the entropy of brown and blue channels are the best predictors for the cell count out of all the four channels. However, even then the correlation is not great with brown entropy achieving 0.456. Considering low correlation coefficients of red and green entropies, it might be best to exclude those variables from the regression.

### d. Histograms

Bellow, I am plotting histograms of each channel. I combine all RGB channels into one histogram because they are following very similar patterns. I only plot the histograms of some of the images trying to find those which show some differences.

In [ ]:
plt.hist(np.reshape(X[2475,:,:,0], (267*267,1)), list(range(0,256, 20)), color = "red")
plt.hist(np.reshape(X[2475,:,:,1], (267*267,1)), list(range(0,256, 20)), color = "green", alpha=0.8)
plt.hist(np.reshape(X[2575,:,:,2], (267*267,1)), list(range(0,256, 20)), color = "blue", alpha=0.5)
plt.title("2475th image's histogram")
plt.show()

In [ ]:
plt.hist(np.reshape(hed_data[2475], (267*267,1)), bins = 20, color = "brown")
plt.title("2457th image's brown histogram")
plt.show()

This particular image is dominated by the blue and brown channels with the red and green being less important. This echoes the mean, variance and entropy results observed above.

Now let's examine another image.

In [ ]:
plt.hist(np.reshape(X[3000,:,:,0], (267*267,1)), bins = list(range(0,256, 20)), color = "red")
plt.hist(np.reshape(X[3000,:,:,1], (267*267,1)), bins = list(range(0,256, 20)), color = "green", alpha=0.9)
plt.hist(np.reshape(X[3000,:,:,2], (267*267,1)), bins = list(range(0,256, 20)), color = "blue", alpha=0.6)
plt.title("3000th image's histograms")
plt.show()

In [ ]:
plt.hist(np.reshape(hed_data[3000], (267*267,1)), bins = 20, color = "brown")
plt.title("3000th image's brown histogram")
plt.show()

In this case, the RGB histograms are mostly overlapping - thus one of those channels should be enough for predicting the cell counts. On the other hand, the brown channel is different and offers an informative feature.

Overall, I think there is enough evidence to discard the green and red channels' variances and entropies from the regression.

### e. PC decomposition

The Principle Component decomposition will be implemented separately for the training and testing data. This is because eigenvectors are formed on the whole data, not just single observations as was the case for all the above features.

Initially, I will try to find only 5 PCs to examine whether those components are any informative. If they all prove to be informative, I will calculate more PCs. In addition, to save time I will use a subset of the training data to test how many components are needed and once that is decided, I will extract the needed number of principal components from the full training and testing data.

In [ ]:
# proof-of-concept for red channel
pca_red = PCA(5, svd_solver='randomized')
projected_red = pca_red.fit_transform(np.reshape(X[:1000,:,:,0],(1000, 267*267)))

In [ ]:
np.sum(pca_red.explained_variance_ratio_)
# 5 PCs explain 32% of the variation

In [ ]:
for i in range(0,4):
    plt.scatter(projected_red[:, i], Y[:1000], edgecolor='none', alpha=0.9, color = "firebrick")
    plt.xlabel('PC 1')
    plt.ylabel( 'Y')
    plt.title("Red PCs  vs cell count")
    plt.show()

In [ ]:
print("PC1 and counts correlation:", np.corrcoef(projected_red[:,0], Y[:1000])[0,1],
     "PC2 and counts correlation:", np.corrcoef(projected_red[:,1], Y[:1000])[0,1],
    "PC3 and counts correlation:", np.corrcoef(projected_red[:,2], Y[:1000])[0,1],
      "PC4 and counts correlation:", np.corrcoef(projected_red[:,3], Y[:1000])[0,1])

From the scatter plots and the correlation coefficients, we see that only the first PC gives meaningful information with the correlation coefficient of the 2nd, 3rd, etc. PCs reaching less than 0.05 in absolute terms

In [ ]:
pca_green = PCA(5, svd_solver="randomized")
projected_green = pca_green.fit_transform(np.reshape(X[:1000,:,:,1],(1000, 267*267)))

In [ ]:
np.sum(pca_green.explained_variance_ratio_)
# 5 PCs explain 34% of a variance.

In [ ]:
for i in range(0,4):
    plt.scatter(projected_green[:, i], Y[:1000], edgecolor='none', alpha=0.9, color = "olivedrab")
    plt.xlabel('PC 1')
    plt.ylabel( 'Y')
    plt.title("Green PCs vs cell count")
    plt.show()

In [ ]:
print("PC1 and counts correlation:", np.corrcoef(projected_green[:,0], Y[:1000])[0,1],
     "PC2 and counts correlation:", np.corrcoef(projected_green[:,1], Y[:1000])[0,1],
    "PC3 and counts correlation:", np.corrcoef(projected_green[:,2], Y[:1000])[0,1],
      "PC4 and counts correlation:", np.corrcoef(projected_green[:,3], Y[:1000])[0,1])

As in the red channel case, only the first principal component of the green channel seems to be informative of the cell count.

In [ ]:
# blue channel PC
pca_blue = PCA(5, svd_solver="randomized")
projected_blue = pca_blue.fit_transform(np.reshape(X[:1000,:,:,2],(1000, 267*267)))

In [ ]:
np.sum(pca_blue.explained_variance_ratio_)
# 5 PCs explain 39% of a variance.

In [ ]:
for i in range(0,4):
    plt.scatter(projected_blue[:, i], Y[:1000], edgecolor='none', alpha=0.9, color = "royalblue")
    plt.xlabel('PC 1')
    plt.ylabel('Y')
    plt.title("Blue PCs vs cell count")
    plt.show()

In [ ]:
print("PC1 and counts correlation:", np.corrcoef(projected_blue[:,0], Y[:1000])[0,1],
     "PC2 and counts correlation:", np.corrcoef(projected_blue[:,1], Y[:1000])[0,1],
    "PC3 and counts correlation:", np.corrcoef(projected_blue[:,2], Y[:1000])[0,1],
      "PC4 and counts correlation:", np.corrcoef(projected_blue[:,3], Y[:1000])[0,1])

Although the blue channel is usually more informative than the red or green, this time, similar to the red and green case, only the first PC is important in explaining cell count.

In [ ]:
pca_brown = PCA(5, svd_solver="randomized")
projected_brown = pca_brown.fit_transform(np.reshape(hed_data[:1000] ,(1000, 267*267)))

In [ ]:
np.sum(pca_brown.explained_variance_ratio_)
# 5 PCs explain 41% of a variance.

In [ ]:
for i in range(0,4):
    plt.scatter(projected_brown[:, i], Y[:1000], edgecolor='none', alpha=0.9, color = "saddlebrown")
    plt.xlabel('PC 1')
    plt.ylabel( 'Y')
    plt.title("Brown PCs vs cell count")
    plt.show()

In [ ]:
print("PC1 and counts correlation:", np.corrcoef(projected_brown[:,0], Y[:1000])[0,1],
     "PC2 and counts correlation:", np.corrcoef(projected_brown[:,1], Y[:1000])[0,1],
    "PC3 and counts correlation:", np.corrcoef(projected_brown[:,2], Y[:1000])[0,1],
      "PC4 and counts correlation:", np.corrcoef(projected_brown[:,3], Y[:1000])[0,1])

Brown channel also showed that only the first PC is informative in predicting the cell counts. Overall, only the first principal component of all the channels seem to be informative with the rest not being informative. This is because the first PCs explain over 20% variation with the rest explaining only a fraction. Furthermore, even though the shapes of RGB PC1 seem the same, their range on the PC1 axis is different. Thus I consider putting all the PC1s into the model. 

Now I will create vectors of PC1 for each channel. Separately for training and testing data.

#### Extracting PC for the training data

In [ ]:
# red channel
pca_red_train = PCA(1, svd_solver='randomized')
projected_red_train = pca_red_train.fit_transform(np.reshape(X[:5841,:,:,0],(5841, 267*267)))

#np.save("projected_red_train", projected_red_train)

In [ ]:
print('Explained variance ratio', pca_red_train.explained_variance_ratio_)
plt.scatter(projected_red_train, Y[:5841], alpha=0.9, color='firebrick')
plt.xlabel('PC 1')
plt.ylabel( 'Y')
plt.title("Red PC1 vs cell count")
plt.show()

In [ ]:
# green channel
pca_green_train = PCA(1, svd_solver='randomized')
projected_green_train = pca_green_train.fit_transform(np.reshape(X[:5841,:,:,1],(5841, 267*267)))

#np.save("projected_green_train", projected_green_train)

In [ ]:
print('Explained variance ratio', pca_green_train.explained_variance_ratio_)
plt.scatter(projected_green_train, Y[:5841], alpha=0.9, color='olivedrab')
plt.xlabel('PC 1')
plt.ylabel( 'Y')
plt.title("Green PC1 vs cell count")
plt.show()

In [ ]:
# blue channel
pca_blue_train = PCA(1, svd_solver='randomized')
projected_blue_train = pca_blue_train.fit_transform(np.reshape(X[:5841,:,:,2],(5841, 267*267)))

# saving to save time in the future
#np.save("projected_blue_train", projected_blue_train)

In [ ]:
print('Explained variance ratio', pca_blue_train.explained_variance_ratio_)
plt.scatter(projected_blue_train, Y[:5841], alpha=0.9, color='royalblue')
plt.xlabel('PC 1')
plt.ylabel( 'Cell count')
plt.title("Blue PC1 vs cell count")
plt.show()

In [ ]:
# Brown channel
pca_brown_train = PCA(1, svd_solver='randomized')
projected_brown_train = pca_brown_train.fit_transform(np.reshape(hed_data[:5841],(5841, 267*267)))

#np.save("projected_brown_train", projected_brown_train)

In [ ]:
print("projected_brown_train", pca_brown_train.explained_variance_ratio_)
plt.scatter(projected_brown_train, Y[:5841], color = "saddlebrown", alpha = 0.9)
plt.title("Brown channel projection")
plt.xlabel('PC 1')
plt.ylabel( 'Cell count')
plt.title("Brown PC1 vs cell count")
plt.show()

#### Test data

Extracting PCs for the test data.

In [ ]:
# red channel
pca_red_test = PCA(1, svd_solver='randomized')
projected_red_test = pca_red_test.fit_transform(np.reshape(X[5841:,:,:,0],(1563, 267*267)))

#np.save("projected_red_test", projected_red_test)
print( "Explained variance ratio:", pca_red_test.explained_variance_ratio_)

In [ ]:
# green channel
pca_green_test = PCA(1, svd_solver='randomized')
projected_green_test = pca_green_test.fit_transform(np.reshape(X[5841:,:,:,1],(1563, 267*267)))

#np.save("projected_green_test", projected_green_test)
print( "Explained variance ratio:", pca_green_test.explained_variance_ratio_)

In [ ]:
# blue channel
pca_blue_test = PCA(1, svd_solver='randomized')
projected_blue_test = pca_blue_test.fit_transform(np.reshape(X[5841:,:,:,2],(1563, 267*267)))

#np.save("projected_blue_test", projected_blue_test)
print( "Explained variance ratio:", pca_blue_test.explained_variance_ratio_)

In [ ]:
# brown channel
pca_brown_test = PCA(1, svd_solver='randomized')
projected_brown_test = pca_brown_test.fit_transform(np.reshape(hed_data[5841:,:,:],(1563, 267*267)))

#np.save("projected_brown_test", projected_brown_test)
print( "Explained variance ratio:", pca_brown_test.explained_variance_ratio_)

### f. My own features

From what I see, when there is a higher concentration of cells (blue and brown), the simple "brownness" can be insufficient because the brown cells in the picture vary in size. Thus, the feature does not capture the relative brownness. To help with this issue, alongside the other  variables, I think it is worth including a ratio of brown cells to blue cells. The blue cells can be extracted via Hematoxylin channel. Then taking the ratio of the "brownness" of the DAB channel to the "blueness" of the Hematoxylin channel could help to separate brown cells from blue ones and also indirectly inform about the size of the brown cells. 

I begin by preprocessing the images and extracting Hematoxylin channel.

In [ ]:
# extracting Hematoxylin channel
hed_blue = rgb2hed(X[:1000,:,:,:])[:,:,:,0]
hed_blue = (hed_blue * 128).astype("int8")

hed_blue_2 = rgb2hed(X[1000:2000])[:,:,:,0]
hed_blue_2 = (hed_blue_2 * 128).astype("int8")

hed_blue = np.vstack((hed_blue, hed_blue_2))

hed_blue_2 = rgb2hed(X[2000:3000])[:,:,:,0]
hed_blue_2 = (hed_blue_2 * 128).astype("int8")

hed_blue = np.vstack((hed_blue, hed_blue_2))

hed_blue_2 = rgb2hed(X[3000:4000])[:,:,:,0]
hed_blue_2 = (hed_blue_2 * 128).astype("int8")

hed_blue = np.vstack((hed_blue, hed_blue_2))

hed_blue_2 = rgb2hed(X[4000:5000])[:,:,:,0]
hed_blue_2 = (hed_blue_2 * 128).astype("int8")

hed_blue = np.vstack((hed_blue, hed_blue_2))

hed_blue_2 = rgb2hed(X[5000:6000])[:,:,:,0]
hed_blue_2 = (hed_blue_2 * 128).astype("int8")

hed_blue = np.vstack((hed_blue, hed_blue_2))

hed_blue_2 = rgb2hed(X[6000:])[:,:,:,0]
hed_blue_2 = (hed_blue_2 * 128).astype("int8")

hed_blue = np.vstack((hed_blue, hed_blue_2))
del(hed_blue_2)
#np.save("hed_blue", hed_blue)

In [ ]:
# calculating the brown/blue ratio

ratio_brown_blue = []
for i in range(0,np.shape(hed_blue)[0]):
    ratio_brown_blue.append(np.mean(hed_data[i,:,:])/(np.mean(hed_blue[i,:,:])))
np.save("ratio_brown_blue", ratio_brown_blue)

In [ ]:
plt.scatter(ratio_brown_blue, Y, color = "goldenrod")
plt.title("Brown/blue vs cell counts")
plt.show()

In [ ]:
print("Correlation between brown/blue and cell counts:", np.corrcoef(ratio_brown_blue, Y)[0,1])

It seems that this feature is a good predictor of the cell counts. It has in absolute terms higher than 50 correlation which is also supported by the scatter plot. In addition, it deals with zero counts slightly better than entropy and variance. Thus, the feature will be added to the regression among other variables.


Another feature which would serve the same purpose as the ratio of brown to blue would be the difference between the brown and blue colours. I expect it to capture similar information as the ratio feature above.

In [ ]:
# calculating difference between brown and blue
difference_blue_brown = []
for i in range(0,np.shape(hed_blue)[0]):
    difference_blue_brown.append(np.mean(hed_blue[i,:,:] - (hed_data[i,:,:])))

In [ ]:
plt.scatter(difference_blue_brown, Y, color = "goldenrod")
plt.title("Blue-brown difference vs cell counts")
plt.show()

In [ ]:
print("Correlation between brown/blue and cell counts:", np.corrcoef(difference_blue_brown, Y)[0,1])

Unfortunately, this feature does not seem to be strong in explaining cell counts. The correlation of 0.29 is much less than that of the ratio (0.53). Thus, I will not include this feature in the initial regression.

## ii) Model implementation

I start by constructing a feature matrix with all the selected features: brown to blue ratio, all average colours, blue and brown variance as well as entropy and the first principal components of all the four channels. I have decided to use pandas so I could have a better look if all the data mergers I do work well.

In [ ]:
features_train = pd.concat([pd.DataFrame(projected_brown_train), 
                            pd.DataFrame(projected_green_train)],axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(projected_red_train)], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(projected_blue_train)], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(average_red[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(average_blue[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(average_green[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(average_brown[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(ratio_brown_blue[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(variance_blue[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(variance_brown[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(entropy_blue[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(entropy_brown[:5841])], axis = 1)

features_train = features_train.to_numpy()

In [ ]:
np.shape(features_train)

For some models such as Support Vector Machine and Ridge Regression, it is best to standardize or normalize the data. This will be done later on.

### a. Linear Regression

Before starting modelling, the cross-validation strategy must be set. The training data should not be validated and tested on the same patient's images since that might lead to model overperformance if it learns specific patterns of a specific patient. Thus, I have decided to manually create 3 folds: Fold 1 containing information from patients 1 to 4 (1935 cases), Fold 2 - patients from 5 to 10 (1949 cases) and Fold 3 - patients 11 to 13 (1957 cases). Consequently, the first training set will include Fold 2 and 3, second, fold 1 and 3 and third - fold 1 and 2. Leaving the remaining fold for validation.

In [ ]:
# creating the function to split data into custom folds
def Folds(X, Y):
    val_1 = X[:1935]
    val_2 = X[1935:3884]
    val_3 = X[3884:]

    val = [val_1, val_2, val_3]

    val_1_labels = Y[:1935]
    val_2_labels = Y[1935:3884]
    val_3_labels = Y[3884:5841]

    val_labels = [val_1_labels, val_2_labels, val_3_labels]

    train_1 = np.vstack((val_2, val_3))
    train_2 = np.vstack((val_1, val_3))
    train_3 = np.vstack((val_1, val_2))

    train = [train_1, train_2, train_3]

    train_1_labels = np.hstack((val_2_labels, val_3_labels))
    train_2_labels = np.hstack((val_1_labels, val_3_labels))
    train_3_labels = np.hstack((val_1_labels, val_2_labels))

    train_labels = [train_1_labels, train_2_labels, train_3_labels]
    
    return train, train_labels, val, val_labels

In [ ]:
train, train_labels, val, val_labels = Folds(features_train, Y)

Now we can run an initial regression with 13 features.

In [ ]:
# creating lists to store the performance metrics
RMSE = []
R2 = []
MAE = []
CORR = []
for i in range(0,3):
    reg = sm.OLS(train_labels[i], train[i])
    results = reg.fit()
    
    # the bellow if condition saves the summary table for all the folds so I could check the coeficient
    # and their significance
    if i == 0:
        res_1 = results.summary()
    elif i == 1:
        res_2 = results.summary()
    elif i == 2:
        res_3 = results.summary()
        
    # Printing AIC score    
    print("AIC:", results.aic)
    
    # predictions on validation
    reg_predict = results.predict(val[i])
    MAE.append(mean_absolute_error(val_labels[i], reg_predict))
    RMSE.append(mean_squared_error(val_labels[i], reg_predict, squared = False))  
    R2.append(r2_score(val_labels[i], reg_predict))
    CORR.append(np.corrcoef(val_labels[i], reg_predict)[0,1])

In [ ]:
print("RMSE:", np.mean(RMSE), "MAE:", np.mean(MAE), "R2:",np.mean(R2), "Correlation:",np.mean(CORR))
# RMSE: 2.935731077584135 MAE: 1.6943052652525477 R2: 0.5796665170172692 Correlation: 0.7741224780008006

In [ ]:
res_1
# looking into the summary of all the folds

The initial regression yielded reasonable results, yet their quality is best to be examined in the light of another model. Looking into the summary tables of all folds, green PC1 and average green seemed to be the most redundant variables, yet removing them did not improve the model based on AIC which means that the extra likelihood the variable gives fully balances out the penalty incurred with increasing model complexity. However, the RMSE went down to 2.8026. Nonetheless, when I have added extra variables in the next part of OLS analysis, the green entropy and variance appeared to be informative and improving the performance measures. Hence I decided to keep these two variables in. 

Note, the regression output signals about the multicollinearity issues, yet since our purpose is prediction, we can ignore that.


As discussed above, I have decided to try out the before excluded variables in - green and red entropy as well as variance. The updated regression is shown below.

In [ ]:
features_train = pd.concat([pd.DataFrame(features_train), pd.DataFrame(entropy_red[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(entropy_green[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(variance_red[:5841])], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(variance_green[:5841])], axis = 1)

features_train = features_train.to_numpy()
#np.save("features_train", features_train)

In [ ]:
np.shape(features_train)

In [ ]:
train, train_labels, val, val_labels = Folds(features_train, Y)

In [ ]:
RMSE = []
R2 = []
MAE = []
CORR = []
for i in range(0,3):
    reg = sm.OLS(train_labels[i], train[i])
    results = reg.fit()
    
     # the bellow if condition saves the summary table for all the folds so I could check the coeficient
    # and their significance
    if i == 0:
        res_1 = results.summary()
    elif i == 1:
        res_2 = results.summary()
    elif i == 2:
        res_3 = results.summary()
        
    # Printing AIC score    
    print("AIC:", results.aic)

    # predictions on validation
    reg_predict = results.predict(val[i])
    MAE.append(mean_absolute_error(val_labels[i], reg_predict))
    RMSE.append(mean_squared_error(val_labels[i], reg_predict, squared = False))  
    R2.append(r2_score(val_labels[i], reg_predict))
    CORR.append(np.corrcoef(val_labels[i], reg_predict)[0,1])

AIC scores fell meaning that the likelihood of the data was improved even after penalizing the model complexity. 

In [ ]:
print("RMSE:", np.mean(RMSE),"MAE:", np.mean(MAE), "R2:", np.mean(R2), "Correlation:", np.mean(CORR))
# Much better. Look closer to coefficients
#RMSE: 2.7500784188753076 MAE: 1.6472135884284327 R2: 0.6333676611925863 Correlation: 0.8213688088261231

This model yielded better results meaning that the extra variables were informative. RMSE fell by roughly 0.18, MAE by 0.04, R2 increased by 5 percentage points and the correlation by approximately 4.5. 

Now I examine the individual variables.

In [ ]:
res_3

The significance of the variables is very sample dependent, thus I decided not to remove anything. I have also tried to add another feature I made - the difference between brown and blue - but it increased AIC and yielded worse validation performance measures. 

Another thing worth trying is to include quadratic terms of the Principal components variables. This is because looking at the PCs plots above, they all exhibit slightly quadratic shape, thus the quadratic components might help to employ the data better. 

In [ ]:
# adding quadratic PC variables to the dataset
features_train = pd.concat([pd.DataFrame(features_train), pd.DataFrame(np.square(projected_brown_train[:5841]))], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(np.square(projected_red_train[:5841]))], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(np.square(projected_green_train[:5841]))], axis = 1)
features_train = pd.concat([features_train, pd.DataFrame(np.square(projected_blue_train[:5841]))], axis = 1)

features_train = features_train.to_numpy()

In [ ]:
train, train_labels, val, val_labels = Folds(features_train, Y)

In [ ]:
# Running the model again
RMSE = []
R2 = []
MAE = []
CORR = []
for i in range(0,3):
    reg = sm.OLS(train_labels[i], train[i])
    results = reg.fit()
    
     # the bellow if condition saves the summary table for all the folds so I could check the coeficient
    # and their significance
    if i == 0:
        res_1 = results.summary()
    elif i == 1:
        res_2 = results.summary()
    elif i == 2:
        res_3 = results.summary()
        
    # Printing AIC score    
    print("AIC:", results.aic)

    # predictions on validation
    reg_predict = results.predict(val[i])
    MAE.append(mean_absolute_error(val_labels[i], reg_predict))
    RMSE.append(mean_squared_error(val_labels[i], reg_predict, squared = False))  
    R2.append(r2_score(val_labels[i], reg_predict))
    CORR.append(np.corrcoef(val_labels[i], reg_predict)[0,1])
# AIC is better    

In [ ]:
print("RMSE:",np.mean(RMSE), "MAE:", np.mean(MAE), "R2:", np.mean(R2), "Correlation", np.mean(CORR))
# Worse that without quadratic terms
# RMSE: 2.8227556858033984 MAE: 1.691787904988676 R2: 0.6142554617774446 Correlation 0.8066181842408485

Even though the quadratic variables improved AIC meaning that they increased the likelihood, the validation performance metrics appeared to be worse. This signals that the quadratic variables help to fit the model to the data better yet results in some degree of overfitting. Thus, for the testing, I have decided to use the data without these quadratic terms. 

Note, I have also tried standardizing and normalizing the data, yet the results appeared to be worse. Thus saving our time and space, I did not include this in the analysis.

#### Testing OLS

First, I will remove the quadratic terms from the training data, construct the test feature matrix, train the model on the whole data and generate predictions.

In [ ]:
# removing quadratic terms
features_train = features_train[:,:17]
np.shape(features_train)

In [ ]:
# Creating testing data
features_test = pd.concat([pd.DataFrame(projected_brown_test), 
                            pd.DataFrame(projected_green_test)],axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(projected_red_test)], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(projected_blue_test)], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(average_red[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(average_blue[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(average_green[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(average_brown[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(ratio_brown_blue[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(variance_blue[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(variance_brown[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(entropy_blue[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(entropy_brown[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(entropy_red[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(entropy_green[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(variance_red[5841:])], axis = 1)
features_test = pd.concat([features_test, pd.DataFrame(variance_green[5841:])], axis = 1)

features_test = features_test.to_numpy()
np.save("features_test", features_test)

In [ ]:
np.shape(features_test)

In [ ]:
# training OLS based on the full training data
reg_train = sm.OLS(Y[:5841], features_train)
results = reg_train.fit()
# predicting based on the testing data
reg_predict_test = results.predict(features_test) 
print("AIC:", results.aic)
# AIC: 27311.223376278205

In [ ]:
print("RMSE:", mean_squared_error(Y[5841:], reg_predict_test, squared = False),
      "MAE:", mean_absolute_error(Y[5841:], reg_predict_test),
     "R2:", r2_score(Y[5841:], reg_predict_test),
     "Correlation:", np.corrcoef(Y[5841:], reg_predict_test)[0,1])
# RMSE: 4.588091148314126 MAE: 3.8287126314793585 R2: 0.19787807294349735 Correlation: 0.7920188573518981

In [ ]:
# true against residuals plot
plt.scatter(Y[5841:], (reg_predict_test - Y[5841:]) )
plt.xlabel("True counts")
plt.ylabel("Residuals")

Overall, the performance is not good and can only serve as a benchmark. From the scatter plot we can see that the biggest issue is with the zero counts as well as large counts. Whilst zero true counts are being overestimated, the large counts are being underestimated. Looking closer into the predicted counts' data, we can see that the model predicts negative counts for some observations. This is a big limitation of OLS and we better focus on the models which can prevent negative predictions from happening.

### Multilayer perceptron

I have started with a multilayer perceptron having 2 hidden layers with a sigmoid activation function and a linear outer layer. I then compared the performance with ReLU and ReLU proved to be a superior network for this problem. I tried a few regularization methods - max norm, dropouts, batch normalization, L2 regularization. Since this MLP is a relatively small network, some of these regularization methods resulted in the over-regularization when the loss of the training data was consistently above that of the validation. I thus relaxed regularization and stayed with L2 regularization only. 

I started running the network for 400 epochs and observed that the period where all of the folds perform the best is around the 100th epoch. At the epoch 150, some of the folds indicated validation set divergence. The loss function graphs are available bellow the MLP. 

In [ ]:
train, train_labels, val, val_labels = Folds(features_train, Y)

In [ ]:
np.shape(features_train)

In [ ]:
# model 3 - ReLU model for 400 epochs

model3 = Sequential()
model3.add(Dense(13,input_dim=np.shape(features_train)[1],
                 kernel_initializer = 'uniform', activation='relu', 
                 kernel_regularizer=regularizers.l2(0.1)))
model3.add(Dense(8,kernel_initializer = 'uniform', activation='relu', 
                 kernel_regularizer=regularizers.l2(0.1)))
model3.add(Dense(4, kernel_initializer = 'uniform', activation='relu',
                kernel_regularizer=regularizers.l2(0.1)))
model3.add(Dense(1,kernel_initializer = 'uniform', activation='relu'))
model3.compile(loss = 'MSE',optimizer='adam',metrics=['MAE'])

# using the same train/validation folds as in the OLS.
for i in range(0,3):
  if i == 0:
    # creating checkpointer to store the results
    checkpoint_path = "training_1/3rd_model_1.ckpt" 
    checkpoint_dir = os.path.dirname(checkpoint_path)  
    cp_model3 = ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only = True,
                                monitor = 'val_loss')
    history3_1 = model3.fit(train[i],train_labels[i], nb_epoch = 400, 
             batch_size = 32, callbacks = [cp_model3],
             validation_data = (val[i], val_labels[i]))
  elif i == 1:
    # creating checkpointer to store the results
    checkpoint_path = "training_1/3rd_model_2.ckpt"
    heckpoint_dir = os.path.dirname(checkpoint_path)  
    cp_model3 = ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only = True,
                                monitor = 'val_loss')
    history3_2 = model3.fit(train[i], train_labels[i], nb_epoch = 400, 
             batch_size = 32, callbacks = [cp_model3],
             validation_data = (val[i], val_labels[i]))
  elif i == 2:
    # creating checkpointer to store the results
    checkpoint_path = "training_1/3rd_model_3.ckpt"
    heckpoint_dir = os.path.dirname(checkpoint_path)  
    cp_model3 = ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only = True,
                                monitor = 'val_loss')
    history3_3 = model3.fit(train[i],train_labels[i], nb_epoch = 400, 
             batch_size = 32, callbacks = [cp_model3],
             validation_data = (val[i], val_labels[i]))

In [ ]:
# plotting validation vs testing loss function for the first fold
plt.plot(history3_1.history['loss'][:])
plt.plot(history3_1.history['val_loss'][:])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plotting validation vs testing loss function for the second fold
plt.plot(history3_2.history['loss'][:])
plt.plot(history3_2.history['val_loss'][:])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plotting validation vs testing loss function for the third fold
plt.plot(history3_3.history['loss'][:])
plt.plot(history3_3.history['val_loss'][:])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Bellow, I will be running the network for 75 epochs for each fold. I have chosen a batch size of 36 because smaller batch sizes resulted in severe fluctuations of the loss curves which made it difficult to spot convergence and divergence. 

Besides, since the neural networks are stochastic, I ran the same neural network a few times to get a better look into the average performance of the model on the validation set. 

In [ ]:
# reruning the network with 75 epochs
model3 = Sequential()
model3.add(Dense(13,input_dim=np.shape(features_train)[1],
                 kernel_initializer = 'uniform', activation='relu', 
                 kernel_regularizer=regularizers.l2(0.2)))
model3.add(Dense(8,kernel_initializer = 'uniform', activation='relu', 
                 kernel_regularizer=regularizers.l2(0.2)))
model3.add(Dense(4, kernel_initializer = 'uniform', activation='relu',
                kernel_regularizer=regularizers.l2(0.2)))
model3.add(Dense(1,kernel_initializer = 'uniform', activation='relu'))
model3.compile(loss = 'MSE',optimizer='adam',metrics=['MAE'])

# creating lists to save validation results.
RMSE_m3 = []
MAE_m3 = []
R2_m3 = []
CORR_m3 = []

# using the same train/validation folds as in the OLS.
for i in range(0,3):
  if i == 0:
    # creating checkpointer to store the results
    checkpoint_path = "training_1/3rd_model_1.ckpt" 
    checkpoint_dir = os.path.dirname(checkpoint_path)  
    cp_model3 = ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only = True, 
                                monitor = 'val_loss')
    history3_1 = model3.fit(train[i],train_labels[i], nb_epoch = 75, 
             batch_size = 32, callbacks = [cp_model3],
             validation_data = (val[i], val_labels[i]))
  elif i == 1:
    # creating checkpointer to store the results
    checkpoint_path = "training_1/3rd_model_2.ckpt"
    heckpoint_dir = os.path.dirname(checkpoint_path)  
    cp_model3 = ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only = True,
                                monitor = 'val_loss')
    history3_2 = model3.fit(train[i], train_labels[i], nb_epoch = 75, 
             batch_size = 32, callbacks = [cp_model3],
             validation_data = (val[i], val_labels[i]))
  elif i == 2:
    # creating checkpointer to store the results
    checkpoint_path = "training_1/3rd_model_3.ckpt"
    heckpoint_dir = os.path.dirname(checkpoint_path)  
    cp_model3 = ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only = True,
                                monitor = 'val_loss')
    history3_3 = model3.fit(train[i],train_labels[i], nb_epoch = 75, 
             batch_size = 32, callbacks = [cp_model3],
             validation_data = (val[i], val_labels[i]))

  # getting validation predictions based on the last epochs weights for each fold
  predicted = model3.predict(val[i])
  RMSE_m3.append(mean_squared_error(val_labels[i], predicted, squared= False))   
  MAE_m3.append(mean_absolute_error(val_labels[i], predicted))
  R2_m3.append(r2_score(val_labels[i], predicted))  
  CORR_m3.append(np.corrcoef(val_labels[i], np.reshape(predicted, (1, np.shape(predicted)[0])))[0,1])

In [ ]:
# plotting validation vs testing loss function for the first fold
plt.plot(history3_1.history['loss'][:])
plt.plot(history3_1.history['val_loss'][:])

In [ ]:
# plotting validation vs testing loss function for the second fold
plt.plot(history3_2.history['loss'][:])
plt.plot(history3_2.history['val_loss'][:])

In [ ]:
# plotting validation vs testing loss function for the third fold
plt.plot(history3_3.history['loss'][:])
plt.plot(history3_3.history['val_loss'][:])
# this folds must be easy, because for this fold only validation and testing loss curves overlap a lot.

The first fold indicates the most  exemplary convergence whilst the second and third folds do not indicate a lot of learning. Looking into the graphs I believe this is the case because even with the initial weights, the networks generates a relatively low error (training RMSE of around 2 for both folds). Regarding the third fold where the validation loss is bellow that of training, it makes me conclude that the third validation set must be "easy".

In [ ]:
print( "RMSE:", np.mean(RMSE_m3), "MAE:", np.mean(MAE_m3), "R2:", np.mean(R2_m3), 
      "Correlation:",np.mean(CORR_m3))

# RMSE: 2.448540108099614 MAE: 1.3937737848963156 R2: 0.7086428615153176 Correlation: 0.8693118140748322
# RMSE: 2.4865251758363804 MAE: 1.3799156730281288 R2: 0.6996322716447878 Correlation: 0.8715614647843276
# RMSE: 2.1417990401852003 MAE: 1.2194835234270782 R2: 0.776113144433234 Correlation: 0.9069356959996351
# RMSE: 2.1675515158649845 MAE: 1.3043220002001195 R2: 0.7720061203151826 Correlation: 0.8976773434131515
# RMSE: 2.4062047717635333 MAE: 1.453062900701184 R2: 0.7101143472110184 Correlation: 0.8946536289511385

In [ ]:
m3_summary = np.array(
[[ 2.448540108099614, 1.3937737848963156, 0.7086428615153176 ,0.8693118140748322],
[2.4865251758363804 , 1.3799156730281288 , 0.6996322716447878 , 0.8715614647843276],
[ 2.1417990401852003 ,1.2194835234270782 , 0.776113144433234 , 0.9069356959996351],
[ 2.1675515158649845 , 1.3043220002001195 , 0.7720061203151826 ,0.8976773434131515],
[ 2.4062047717635333 ,1.453062900701184 ,0.7101143472110184 ,0.8946536289511385]])

In [ ]:
print("RMSE:", np.mean(m3_summary[:,0]), "MAE:", np.mean(m3_summary[:,1]), 
      "R2:", np.mean(m3_summary[:,2]), "Correlation", np.mean(m3_summary[:,3]))
# RMSE: 2.3301241223499423 MAE: 1.3501115764505651 R2: 0.733301749023908 Correlation 0.888027989444617

print("RMSE variance:", np.var(m3_summary[:,0]), "MAE variance:", np.var(m3_summary[:,1]), 
      "R2 variance:", np.var(m3_summary[:,2]), "Correlation variance", np.var(m3_summary[:,3]))
#RMSE variance: 0.021233617875919535 MAE variance: 0.00651080426280465 R2 variance: 0.0011220387953846586 
#Correlation variance 0.00022319043058231704

Multilayer Perceptron definitely outperforms OLS at least on the validation set. RMSE decreased by on average 0.45, MAE fell by 0.32, R2 increased by 11 percentage points and correlation - by roughly 7. Variance remains relatively low and builds confidence in the results. 

Note, I have also tried to standardize and normalize the data for the MLP but the results appeared to be worse. For the same reason as before - saving your and my time - I will not include the analysis of the preprocessed data.

Now, we need to see MLPs testing performance.

#### Testing MLP

In [ ]:
model3 = Sequential()
model3.add(Dense(13,input_dim=np.shape(features_train)[1],
                 kernel_initializer = 'uniform', activation='relu', 
                 kernel_regularizer=regularizers.l2(0.2)))
model3.add(Dense(8,kernel_initializer = 'uniform', activation='relu', 
                 kernel_regularizer=regularizers.l2(0.2)))
model3.add(Dense(4, kernel_initializer = 'uniform', activation='relu',
                kernel_regularizer=regularizers.l2(0.2)))
model3.add(Dense(1,kernel_initializer = 'uniform', activation='relu'))
model3.compile(loss = 'MSE',optimizer='adam',metrics=['MAE'])


checkpoint_path = "training_1/final_MLP.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)  
cp_model3 = ModelCheckpoint(checkpoint_path, save_weights_only=True,  verbose=1,
                              monitor = 'MSE')
history_test = model3.fit(features_train,Y[:5841], nb_epoch = 75, batch_size = 32, 
                          callbacks = [cp_model3])

In [ ]:
# plotting training loss function
plt.plot(history_test.history['loss'])
# steady and reasuring convergence. 

In [ ]:
# predicting based on the training set
predictions_test = model3.predict(features_test)

print("RMSE:", mean_squared_error(Y[5841:], predictions_test, squared = False),
      "MAE:", mean_absolute_error(Y[5841:], predictions_test),
     "R2:", r2_score(Y[5841:], predictions_test),
     "Correlation:", np.corrcoef(Y[5841:], np.reshape(predictions_test,
                                                      (1, np.shape(predictions_test)[0])))[0,1])

# RMSE: 2.6873412156480345 MAE: 1.39978451512978 R2: 0.7248167850851046 Correlation: 0.858725725571641
# RMSE: 3.218160973821551 MAE: 1.6606015762170003 R2: 0.6053684147560368 Correlation: 0.8638262287048433
# RMSE: 2.6120960289700195 MAE: 1.33473918143176 R2: 0.7400112283512154 Correlation: 0.868681345290561
# RMSE: 3.0623111200234505 MAE: 1.5725380469039703 R2: 0.6426655094933946 Correlation: 0.8130529428367174
# RMSE: 2.7921602825129033 MAE: 1.3609376257227082 R2: 0.7029312256022222 Correlation: 0.8455899133

In [ ]:
m3_test_summary = np.array(
[[ 2.6873412156480345 , 1.39978451512978 , 0.7248167850851046 , 0.858725725571641],
[ 3.218160973821551 ,1.6606015762170003 , 0.6053684147560368 ,0.8638262287048433],
[ 2.6120960289700195 , 1.33473918143176 , 0.7400112283512154 , 0.868681345290561],
[ 3.0623111200234505 , 1.5725380469039703 ,0.6426655094933946 , 0.8130529428367174],
[ 2.7921602825129033 , 1.3609376257227082 , 0.7029312256022222 , 0.8455899133]])

In [ ]:
print("RMSE:", np.mean(m3_test_summary[:,0]), "MAE:", np.mean(m3_test_summary[:,1]), 
      "R2:", np.mean(m3_test_summary[:,2]), "Correlation", np.mean(m3_test_summary[:,3]))
# RMSE: 2.8744139241951916 MAE: 1.4657201890810438 R2: 0.6831586326575947 Correlation 0.8499752311407527

print("RMSE variance:", np.var(m3_test_summary[:,0]), "MAE variance:", np.var(m3_test_summary[:,1]), 
      "R2 variance:", np.var(m3_test_summary[:,2]), "Correlation variance:", np.var(m3_test_summary[:,3]))
#RMSE variance: 0.05280798566709645 MAE variance: 0.016374346570017247 
#R2 variance: 0.0026099171514585642 Correlation variance: 0.00040016527581236683

In testing data, MLP performs noticeably better than OLS. RMSE fell by 1.72, MAE fell by approximately 2,363, R2 rose by 48 percentage points and correlation rose by approximately 6 points. It is therefore sure, that MLP exploits the information from the extracted features much more efficiently than OLS. 

Nonetheless, MLP still makes an error which is on average equal to 1.5 counts (MAE). Whilst it is not a lot, it is good to examine which parts the network finds especially difficult. To do this, I plot residual versus true counts scatter plot below.


In [ ]:
# true counts vs residuals plot
plt.scatter(Y[5841:], (np.reshape(predictions_test, (1, np.shape(predictions_test)[0])) - Y[5841:]))

Compared to OLS, MLP deals better with the zero counts data. This is because the model never predicts negative counts (due to the ReLU activation function which sets all negative values to zero). Overall, MLP works especially well for low to mid-range cell counts - the error is the smallest for the 0-10 cell count data. OLS struggled with low to mid-range counts too. Unfortunately, as with OLS, MLP struggles to accurately count the high cell data. From the scatter plot above, it seems that the model tends to underestimate the number of cells for the high count data. This suggests that we need to extract features which are characteristic to the high cell count data. Better examination of the images or CNN could help with this issue. 

### Ridge Regression

Ridge regression requires optimizing for the alpha parameters. Since I have custom folds, I will not use sklearn GridSearch but instead, write a barebones grid search to incorporate the custom folds. 

In [ ]:
# setting the parameter spaxe
parameters_ridge = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# empty list to incorporate the mean values across all three folds.
RMSE_ridge = []

for a in parameters_ridge:
    # calling ridge regression
    ridge = Ridge(a)
    # list to hold RMSE scores of each fold - overritten each "alpha" loop
    RMSE_temp = []
    # loop against the 3 folds
    for i in range(0,3):
        ridge.fit(train[i], train_labels[i])
        pred = ridge.predict(val[i])
        RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
    # getting the mean accross all folds for each alpha parameter    
    RMSE_ridge.append(np.mean(RMSE_temp))
        

In [ ]:
RMSE_ridge

It seems that the alpha of 0.1 gives the highest validation performance indicating that less regularization is preferred. The RMSE score of 2.786 is slightly higher than the validation score achieved by OLS but this is expected due to the regularization. 

Ridge regression might benefit from the preprocessing and thus bellow I explore the effects of standardization and normalization.

In [ ]:
features_norm = preprocessing.normalize(features_train, norm="l2")
train, train_labels, val, val_labels = Folds(features_norm, Y)

In [ ]:
# setting the parameter space
parameters_ridge = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# empty lis to incorporate the mean values across all three folds.
RMSE_ridge = []

for a in parameters_ridge:
    # calling ridge regression
    ridge = Ridge(a)
    # list to hold RMSE scores of each fold - overritten each "alpha" loop
    RMSE_temp = []
    # loop against the 3 folds
    for i in range(0,3):
        ridge.fit(train[i], train_labels[i])
        pred = ridge.predict(val[i])
        RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
    # getting the mean accross all folds for each alpha parameter    
    RMSE_ridge.append(np.mean(RMSE_temp))        

In [ ]:
RMSE_ridge

Results with the normalized data are worse than with the un-processed data.

In [ ]:
scaler = preprocessing.StandardScaler()
features_scale = scaler.fit_transform(features_train)
train, train_labels, val, val_labels = Folds(features_scale, Y)

In [ ]:
# setting the parameter space
parameters_ridge = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# empty lis to incorporate the mean values across all three folds.
RMSE_ridge = []

for a in parameters_ridge:
    # calling ridge regression
    ridge = Ridge(a)
    # list to hold RMSE scores of each fold - overritten each "alpha" loop
    RMSE_temp = []
    # loop against the 3 folds
    for i in range(0,3):
        ridge.fit(train[i], train_labels[i])
        pred = ridge.predict(val[i])
        RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
    # getting the mean accross all folds for each alpha parameter    
    RMSE_ridge.append(np.mean(RMSE_temp))
        

In [ ]:
RMSE_ridge

The results using standardized data are better than using normalized data, yet worse than using un-processed data. Thus, for the ridge regression, I have decided to use the un-processed data with alpha = 0.1.

#### Testing Ridge Regression

In [ ]:
ridge_test = Ridge(alpha = 0.1).fit(features_train, Y[:5841])

In [ ]:
print(ridge_test.coef_)

In [ ]:
ridge_predict = ridge_test.predict(features_test)

In [ ]:
print("RMSE:", mean_squared_error(Y[5841:], ridge_predict, squared = False),
      "MAE:", mean_absolute_error(Y[5841:], ridge_predict),
     "R2:", r2_score(Y[5841:], ridge_predict),
      "Correlation:", np.corrcoef(Y[5841:], ridge_predict)[0,1])
# RMSE: 3.776070401818422 MAE: 2.8368163257488477 R2: 0.45667901445475345 Correlation: 0.7989085675534381

First of all, Ridge regression yielded inferior results to MLP. When it comes to OLS, the regularization did manage to improve the results. RMSE fell by 0.81, MAE fell by roughly 1, R2 rose by approximately 25 percentage points whilst the correlation rose by a marginal 0.006. 

In [ ]:
plt.scatter(Y[5841:], (ridge_predict - Y[5841:]))
plt.title("True counts vs residuals")
plt.show()

From the true counts vs predicted scatter plot, we can see that the model tends to underestimate the number of counts and deals with the zero counts worse than MLP. Thus MLP so far stands as the best model.

### Support Vector Regression

Support Vector Regression has a few parameters which need to be tuned in. For this reason, I will employ Grid Search. However, since I have custom training and testing folds, I will again write a barebones grid search to incorporate my folds. Furthermore, Support Vector Regression (especially RBF kernel and linear models with L2, L1 losses) usually assume that all features are standardized. Thus, I will preprocess the data in two different way for the SVR model analysis -  either normalize it or standardize it. I will start from analysing linear kernel followed by polynomial and RBF kernels.

Starting from the linear SVR, I am using LinearSVR due to it being more efficient than the standard SVR from sklearn library. In addition, since the data has a higher number of observations than features, I set the algorithm to solve a primal optimization problem. Consequently, I opt for L2 loss. 

I first start with standardized data and then try normalized data (it is not centred around zero but in practice works well). The Linear SVR with non-preprocessed data did not converge even after adjusting the tolerance and maximum iterations, thus I will not be reporting it.

In [ ]:
# getting standardized folds again
train, train_labels, val, val_labels = Folds(features_scale, Y)

In [ ]:
# setting the parameter space for linear kernel
C_param = [0.1, 1, 10, 100, 1000]
epsilon_param = [0.5, 1, 2]

# empty lis to incorporate the mean values across all three folds.
SVR_linear = []

# Focus on linear kernel only. 

# loop over C parameter
for c in C_param:
    for e in epsilon_param:
    # list to hold individual for RMSE values
        RMSE_temp = []
        # loop over the three folds defined above. 
        for i in range(0,3):
            svr = LinearSVR(C = c, epsilon = e, dual = False, loss = 'squared_epsilon_insensitive')
            svr.fit(train[i], train_labels[i])
            # predict based on validation
            pred = svr.predict(val[i])
            # calculate RMSE of the validation fold
            RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
        # take the average RMSE over all three validation folds.     
        SVR_linear.append(np.mean(RMSE_temp))

In [ ]:
# Result for SVR with a linear kernel. 
SVR_linear

In [ ]:
# getting normalized folds again
train, train_labels, val, val_labels = Folds(features_norm, Y)

In [ ]:
# setting the parameter space for linear kernel
C_param = [0.1, 1, 10, 100 , 1000]
epsilon_param = [0.5, 1, 2]

# empty lis to incorporate the mean values across all three folds.
SVR_linear = []

# Focus on linear kernel only. 

# loop over C parameter
for c in C_param:
    for e in epsilon_param:
    # list to hold individual for RMSE values
        RMSE_temp = []
        # loop over the three folds defined above. 
        for i in range(0,3):
            svr = LinearSVR(C = c, epsilon = e, dual = False, loss = 'squared_epsilon_insensitive')
            svr.fit(train[i], train_labels[i])
            # predict based on validation
            pred = svr.predict(val[i])
            # calculate RMSE of the validation fold
            RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
        # take the average RMSE over all three validation folds.     
        SVR_linear.append(np.mean(RMSE_temp))

In [ ]:
SVR_linear

As expected, the standardized data works better in the linear SVR case. The lowest result of 2.851 with the standardized data was achieved with C = 1000 and epsilon 0.5. Normalized data, in turn, yielded the lowest score of 3.670. I will thus from now on focus on the standardized data only.

Now, I will test the polynomial kernel.

In [ ]:
# getting standardized folds again
train, train_labels, val, val_labels = Folds(features_scale, Y)

In [ ]:
# setting the parameter space for the polynomial SVR
C_param = [ 0.01, 0.1, 1, 10, 100]
degree_param = [2,3]
epsilon_param = [0.5, 1, 2]

# empty lis to incorporate the mean values across all three folds.
SVR_poly = []

# Focus on polynomial kernel

# loop over the degrees of polynomial -  starting with 2 and 3
for d in degree_param:
    # loop over C parameter
    for c in C_param:
        # loop over epsilon
        for e in epsilon_param:
            # list to hold individual for RMSE values
            RMSE_temp = []
            for i in range(0,3):
                svr = SVR(kernel = 'poly', degree = d, epsilon = e, C = c)
                svr.fit(train[i], train_labels[i])
                # predict based on validation
                pred = svr.predict(val[i])
                # calculate RMSE of the validation fold
                RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
            # take the average RMSE over all three validation folds.       
            SVR_poly.append(np.mean(RMSE_temp))

In [ ]:
# printing out the polynomial SVR results
SVR_poly
# degree = 2, C = 1 and epsilon = 2 gives RMSE of  3.0267288143717423,

SVR with polynomial kernel achieved the lowest RMSE OF 3.025 with the polynomial degree of 2, C = 1 and epsilon 2. However, the linear SVR had achieved a better validation performance and so far remains a stronger candidate. 

I will now only have to examine the RBF kernel.

In [ ]:
# Grid search with a focus on RBF kernel

gamma_param = [ 1e-2, 1e-4, 1, 10]
C_param = [ 0.01, 0.1, 1, 10, 100]
epsilon_param = [0.5, 1, 2]

# empty lis to incorporate the mean values across all three folds.
SVR_rbf = []

# Focus on RBF kernel

# Loop over gamma parameter
for g in gamma_param:
    # loop over C parameter
    for c in C_param:
        # list to hold individual for RMSE values
        for e in epsilon_param:
            RMSE_temp = []
            for i in range(0,3):
                svr = SVR(kernel = 'rbf', gamma = g, epsilon = e, C = c)
                svr.fit(train[i], train_labels[i]) 
                 # predict based on validation
                pred = svr.predict(val[i])
                # calculate RMSE of the validation fold
                RMSE_temp.append(mean_squared_error(val_labels[i], pred, squared = False))
            # take the average RMSE over all three validation folds.       
            SVR_rbf.append(np.mean(RMSE_temp))

In [ ]:
# printing RBF kernel SVR results
SVR_rbf
# Lowest RMSE =  2.240149296447879,

RBF kernel outperforms all the above SVR models. The lowest score of 2.240 was achieved with gamma = 0.01, C = 10 and epsilon = 0.5. This yields a  roughly 0.6 lower RMSE than with the linear kernel. I thus choose RBF with gamma = 0.01, C = 10 and epsilon = 0.5 to be my chosen model for the testing.

#### Testing SVR

In [ ]:
# defining the testing model 
svr = SVR(kernel = 'rbf', gamma = 0.01, C = 10, epsilon = 0.5).fit(features_scale, Y[:5841])

# standardizing the testing data
features_scale_test = scaler.fit_transform(features_test)

svr_predict = svr.predict(features_scale_test)

In [ ]:
print("RMSE:", mean_squared_error(Y[5841:], svr_predict, squared = False),
      "MAE:", mean_absolute_error(Y[5841:], svr_predict),
      "R2:", r2_score(Y[5841:], svr_predict),
      "Correlation:",np.corrcoef(Y[5841:], svr_predict)[0,1])
# RMSE: 3.607931584126985 MAE: 1.942052130388768 R2: 0.5039871800491145 Correlation: 0.729540358418439

In [ ]:
plt.scatter(Y[5841:], (svr_predict - Y[5841:]))
plt.title("True counts vs residuals")

SVR with RBF kernel performed better than the Ridge regression. RMSE fell by roughly 0.17, MAE by 0.896 R2 rose by roughly 4.5 percentage points, yet Correlation fell by 0.069. Yet, it still did not manage to outperform the MLP which stands as the best model considered so far. 

A summary table bellow allows a quick comparison of all the model discussed so far.

### Summary of the results

In [ ]:
data_summary = { " ": ["OLS", "MLP", "Ridge", "SVR"],
                    "RMSE": [mean_squared_error(Y[5841:], reg_predict_test, squared = False),
                             np.mean(m3_test_summary[:,0]),
                            mean_squared_error(Y[5841:], ridge_predict, squared = False),
                            mean_squared_error(Y[5841:], svr_predict, squared = False)],
                  "MAE": [mean_absolute_error(Y[5841:], reg_predict_test),
                          np.mean(m3_test_summary[:,1]),
                        mean_absolute_error(Y[5841:], ridge_predict), 
                        mean_absolute_error(Y[5841:], svr_predict)],
                      "R2": [r2_score(Y[5841:], reg_predict_test),
                              np.mean(m3_test_summary[:,2]),
                            r2_score(Y[5841:], ridge_predict),
                             r2_score(Y[5841:], svr_predict)],
                     "Correlation": [np.corrcoef(Y[5841:], reg_predict_test)[0,1],
                                     np.mean(m3_test_summary[:,3]),
                                     np.corrcoef(Y[5841:], ridge_predict)[0,1],
                                      np.corrcoef(Y[5841:], svr_predict)[0,1]],
                      
                     }

model_summary = pd.DataFrame(data_summary, columns = [' ','RMSE','MAE','R2','Correlation'])

model_summary

## Question 3

The first task is to decide on the cross-validation technique. It would be ideal to use the same technique as in the Question 2, yet it will take a lot more time to do so. Thus, saving time, I have decided to use one fold validation instead. Consequently, I have decided to use the images from patients 1 to 10 (inclusive) for testing and patients 11 to 13 (inclusive) for testing.

In [ ]:
# seeing again the unique counts so I can decide on splits
np.unique(P, return_counts = True)

In [ ]:
# splitting the data
X_train = X[:3884,:,:,:].astype(np.float32)
Y_train = Y[:3884].astype(np.float32)
X_test = X[3884:5841,:,:,:].astype(np.float32)
Y_test = Y[3884:5841].astype(np.float32)

I have started from 2 convolutional layer and 2 hidden layers network all with ReLU activation function and batch size 32. Trying out various numbers of kernels and neurons, I have observed that the training model has achieved the best performance when the number of filters in the second convolutional layer is larger than in the first. However, even if the model was learning the training data very well (MSE dropped to around 1), it was overfitting so that the validation error would remain high. Thus, I proceeded with regularization. Beginning with L2 normalization did not improve the results a lot, this is, I suppose was the case due to the fact that we have rather large images and even the low constant of normalization forces the weights to decay too much. Following that, I have tried max-norm with dropout technique described to be working well in the Srivastava et al. (2014). However, this also did not bring satisfactory results. Lastly, I replaced max-norm constraint by the Batch Normalization. Increasing the drop-out rate to 30% alongside normalizing batches gave me the best results. However, the validation loss graph seemed to be very volatile. This obstructed from seeing the convergence. To solve this issue I increased the batch size to 64 and standardized the data. Unfortunately, whilst this partly solved the problem and the trend became less volatile, the minimum MSE loss achieved on validation was over 5 which is higher than what I have achieved before. Lastly, wanting to tune in my model better I have tried a linear activation function for the outer layer. Both, linear and ReLU activation functions on the outer layer seemed to yield similar results. Linear activation would show signs of convergence around 35th epoch and would yield validation RMSE of 1.8719385 whilst ReLU outer layer would converge around 45th epoch and yield RMSE of 1.868874. Thus, even though it is a marginal decline, I have chosen the model with the ReLU outer layer. The model is thus presented bellow.

In [ ]:
# Defining the final CNN model
model = Sequential()
model.add(Conv2D(18, kernel_size=(3, 3), activation='relu',  
                 input_shape=(267,267,3)))
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Conv2D(36, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Dense(24, activation='relu'))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Dense(1, activation='relu'))


model.compile(loss='MSE',optimizer='adam',metrics=['MAE'])

# storing the model so could retain later.
checkpoint_path = "weights_final_cnn_relu.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)  
cp_model = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', save_weights_only= False,
                              save_best_only=False, verbose=1, mode = "min")
# early stopping so would not waste the time too much
early_stopping = EarlyStopping(monitor='val_loss', patience=40,
    verbose=1, mode='min', baseline=None, restore_best_weights=True)

history1 = model.fit(X_train, Y_train,
                     validation_data= (X_test, Y_test), 
                     batch_size=64, epochs=100, callbacks = [early_stopping, cp_model], verbose=1) 

In [ ]:
# plotting training and validation MSE loss
plt.plot(history1.history['loss'][:])
plt.plot(history1.history['val_loss'][:])
plt.title('model MSE')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# convergence signs between 40 to 60

In [ ]:
# Plotting training and validation MAE loss
plt.plot(history1.history['MAE'][:])
plt.plot(history1.history['val_MAE'][:])
plt.title('model MAE')
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# convergence signs between 40 and 60

In [ ]:
# getting validation results with the best weights
pred_relu = model.predict(X_test)

print("RMSE:", mean_squared_error(Y_test, pred_relu, squared = False),
      "MAE:", mean_absolute_error(Y_test, pred_relu),
      "R2:", r2_score(Y_test, pred_relu),
      "Correlation:", np.corrcoef(Y_test, np.reshape(pred_relu, (1, np.shape(pred_relu)[0])))[0,1])
# 1.868874 1.1523674 0.8116560342960611 0.9009959414135418

The validation performance is very good, better than any other model considered before. However, the validation weights were "hand-picked" here since I could see their loss whilst training. Now let's examine the performance on the testing data.

#### Testing CNN

In [ ]:
# testing the model on unseen data
input_shape = (267, 267, 3)

model = Sequential()
model.add(Conv2D(18, kernel_size=(3, 3), activation='relu',  
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Conv2D(36, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Dense(24, activation='relu'))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.3))
model.add(Dense(1, activation='relu'))


model.compile(loss='MSE',optimizer='adam',metrics=['MAE'])

checkpoint_path = "weights_final_cnn_test.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)  
cp_model = ModelCheckpoint(checkpoint_path, monitor = 'loss',  
                              save_best_only=False, verbose=1, mode = "min")

history1 = model.fit(X[:5841,:,:,:].astype(np.float32), Y[:5841].astype(np.float32), 
                     batch_size=64, epochs=45, 
                     callbacks = [cp_model], verbose=1) 

In [ ]:
# plotting training loss curve
plt.plot(history1.history['loss'][:])
plt.title('model MSE')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
# getting validation performace statistics
model.load_weights("weights_final_cnn_test.hdf5")
pred_relu = model.predict(X[5841:,:,:,:].astype(np.float32))

print("RMSE:", mean_squared_error(Y[5841:].astype(np.float32), pred_relu, squared = False),
      "MAE:",mean_absolute_error(Y[5841:].astype(np.float32), pred_relu),
      "R2:",r2_score(Y[5841:].astype(np.float32), pred_relu),
      "Correlation:", np.corrcoef(Y[5841:].astype(np.float32), np.reshape(pred_relu, (1, np.shape(pred_relu)[0])))[0,1])
# 2.6366537 1.8547359 0.7350996790305697 0.8788127874874672
# 2.653262 1.7509032 0.7317519530679418 0.8724536907598069
# 3.5863793 2.185601 0.509895425576637 0.8444601093429096
# 3.0134172 2.2493796 0.6539850318898193 0.8506449953651958
# 3.5236025  2.448045  0.5269030788416225  0.8031634577696839

In [ ]:
CNN_summary = np.array(
[[2.6366537, 1.8547359, 0.7350996790305697, 0.8788127874874672],
[2.653262, 1.7509032, 0.7317519530679418, 0.8724536907598069],
[3.5863793, 2.185601, 0.509895425576637, 0.8444601093429096],
[3.0134172, 2.2493796, 0.6539850318898193, 0.8506449953651958],
 [3.5236025,  2.448045,  0.5269030788416225,  0.8031634577696839]])

In [ ]:
print("RMSE:", np.mean(CNN_summary[:,0]), "MAE:", np.mean(CNN_summary[:,1]), 
      "R2:", np.mean(CNN_summary[:,2]), "Correlation", np.mean(CNN_summary[:,3]))
# RMSE: 3.0826629399999996 MAE: 2.09773294 R2: 0.6315270336813181 Correlation 0.8499070081450126

print("RMSE variance:", np.var(CNN_summary[:,0]), "MAE variance:", np.var(CNN_summary[:,1]), 
      "R2 variance:", np.var(CNN_summary[:,2]), "Correlation variance:", np.var(CNN_summary[:,3]))
#RMSE variance: 0.1672524497702104 MAE variance: 0.06655489496719841 
#R2 variance: 0.009403421806131901 Correlation variance: 0.0007118139619430679

In [ ]:
plt.scatter(Y[5841:], (np.reshape(pred_relu, (1, np.shape(pred_relu)[0])) - Y[5841:]))
plt.title("CNN true vs residual scatterplot")
plt.show()

The performance on testing data did not bring very good news. Whilst the two first times, CNN managed to yield RMSE of roughly 2.64, the third times it rose well over 3. Consequently, an average performance remains bellow that of Multilayer Perceptron. There can be several possible explanations, why this is the case. Firstly, it seems that the model is learning the training data well, yet it fails to generalize, thus I would need to further investigate regularization techniques. Secondly, it might be better to opt for a steadier model with standardized data since, at least, it seems to yield constant results over the validation set albeit the minimum score remains above that of the unprocessed data. Thirdly, it might be the case that the chosen validation set is "easy". From MLP we can see that if we keep the last third of the data as validation, the testing loss trend sometimes even goes bellow training. If the third set is really too easy, we would need to validate the model on a different set. 

Overall, CNN faces the same difficulties as the previous models. It struggles to recognise zero count data as well as underestimates the high counts. We can explore the feature maps and see what features the model was extracting and maybe understand where the problem of the neural network is.

#### Feature maps

In [ ]:
# summarising the model
model.summary()

In [ ]:
# Choosing an arbitrary image for which feature maps will be generated
plt.imshow(X[6000,:,:,:])

In [ ]:
# plotting the feature maps of the firs convolutional layer.
model_1 = tf.compat.v1.keras.Model(inputs=model.inputs, outputs=model.layers[1].output)

# extracting the feature maps
feature_maps = model_1.predict(np.reshape(X[6000,:,:,:], (1, 267, 267, 3)))

# plot all 16 maps in a 3x6 grid

ix = 1
for _ in range(3):
	for _ in range(6):
		# specify subplot and turn of axis
		ax = plt.subplot(3, 6, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
		ix += 1
# show the figure
plt.show()

In [ ]:
model_2 = tf.compat.v1.keras.Model(inputs=model.inputs, outputs=model.layers[5].output)

feature_maps_2 = model_2.predict(np.reshape(X[6000,:,:,:], (1, 267, 267, 3)))

# plot all 36 maps in a 6x6 grid

ix = 1
for _ in range(6):
	for _ in range(6):
		# specify subplot and turn of axis
		ax = plt.subplot(6, 6, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		plt.imshow(feature_maps_2[0, :, :, ix-1], cmap='gray')
		ix += 1
# show the figure
plt.show()

From the feature maps, it seems that the convolutional kernels are learning to predict the data well. In the first batch of feature maps, we can see that it separates brown cells from the blue cells as well as manages to isolate cells from the background. The same can be seen from the second batch of feature maps. Thus, I am leaning towards the idea that the main issue with this CNN built is not the convolutional layers but the counting. It is therefore likely that if instead of MLP, I have tried some different architecture or even different machine learning techniques, I could have realised a fuller power of these convolutional layers. 

THE END

*What now, José?
The party’s over,
the lights are off,
the crowd’s gone,
the night’s gone cold,
what now, José?* - Carlos Drummond de Andrade 

**References**

Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I. and Salakhutdinov, R., 2014. Dropout: a simple way to prevent neural networks from overfitting. The journal of machine learning research, 15(1), pp.1929-1958.